In [1]:
!pip3 install -q datasets peft accelerate evaluate scikit-learn rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 w

In [2]:
!pip3 install -q --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 92.9 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
from datasets import load_dataset
import evaluate
from transformers import (
    GPT2Config,
    GPT2TokenizerFast,
    GPT2PreTrainedModel,
    GPT2Model,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    get_linear_schedule_with_warmup,
)
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

MODEL_NAME    = "gpt2"
MAX_LENGTH    = 256
BATCH_SIZE    = 8
LEARNING_RATE = 2e-5
NUM_EPOCHS    = 3
OUTPUT_DIR    = "sent_sft"
PATIENCE      = 2       # for early stopping
WARMUP_RATIO  = 0.1

train_raw = load_dataset("amazon_polarity", split="train[:200000]")
val_raw   = load_dataset("amazon_polarity", split="test[:20000]")

metric    = evaluate.load("accuracy")
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
PAD_ID    = tokenizer.pad_token_id

# ── 3) Preprocess function ────────────────────────────────────────────────────
def preprocess(batch):
    enc = tokenizer(
        batch["content"],
        truncation=True,
        max_length=MAX_LENGTH,
    )
    enc["labels"] = batch["label"]
    return enc

train_enc = train_raw.map(preprocess, batched=True, remove_columns=train_raw.column_names)
val_enc   = val_raw.map(preprocess,   batched=True, remove_columns=val_raw.column_names)

data_collator = DataCollatorWithPadding(tokenizer)


2025-05-27 07:44:48.418005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748331888.608205      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748331888.662055      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [4]:
class GPT2ForSentiment(GPT2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.transformer = GPT2Model(config)
        self.dropout     = nn.Dropout(0.1)
        self.classifier  = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None):
        out = self.transformer(input_ids, attention_mask=attention_mask)
        last_h = out.last_hidden_state            
        mask   = attention_mask.unsqueeze(-1)     
        pooled = (last_h * mask).sum(1) / mask.sum(1)  
        x      = self.dropout(pooled)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)

        return {"loss": loss, "logits": logits}

In [5]:
config = GPT2Config.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    pad_token_id=PAD_ID,
)
model = GPT2ForSentiment.from_pretrained(MODEL_NAME, config=config)
model.resize_token_embeddings(len(tokenizer))

device = (
    "mps" if torch.backends.mps.is_available() else
    "cuda" if torch.cuda.is_available() else "cpu"
)
model.to(device)
print(f"Using device: {device}")

total_steps = (len(train_enc) // BATCH_SIZE) * NUM_EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSentiment were not initialized from the model checkpoint at gpt2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda


In [6]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_strategy="epoch",
    logging_steps=200,
    fp16=True,
    report_to=[],
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,
)

class SchedulerCallback(EarlyStoppingCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        self.scheduler = get_linear_schedule_with_warmup(
            self.model.optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=total_steps,
        )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=val_enc,
    data_collator=data_collator,
    compute_metrics=lambda p: {"accuracy": metric.compute(predictions=np.argmax(p.predictions, -1), references=p.label_ids)["accuracy"]},
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
)

trainer.train()
eval_res = trainer.evaluate()
print(f"Validation accuracy: {eval_res['eval_accuracy']:.4f}")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.230100,0.211605,0.940600
2,0.160800,0.202438,0.948600
3,0.119500,0.235993,0.948900


Validation accuracy: 0.9486


In [7]:
preds_out = trainer.predict(val_enc)
preds, labels = np.argmax(preds_out.predictions, -1), preds_out.label_ids

print("\nClassification Report:")
print(classification_report(labels, preds, target_names=["negative","positive"]))


Classification Report:
              precision    recall  f1-score   support

    negative       0.94      0.95      0.95      9786
    positive       0.95      0.94      0.95     10214

    accuracy                           0.95     20000
   macro avg       0.95      0.95      0.95     20000
weighted avg       0.95      0.95      0.95     20000

